In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
# import polars as pl
# import duckdb as dd
from tqdm import tqdm
"""import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes"""
# from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import tensorflow_io as tfio
from tensorflow import keras
from tensorflow.python.keras import backend as K
from joblib import Parallel, delayed

In [2]:
import tensorflow as tf
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  2


In [4]:
Test = False
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 10
    config['end'] = 460
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    #studies = os.listdir(config['root_file_path'])
    test_dict = {}
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    studies = os.listdir(config['root_file_path'])
    test_dict = {}
    
for study in studies:
    image_files = []
    for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))
            
print(len(test_dict))

450


In [ ]:
"""for st in test_dict.keys():
    print("number of images in the study {0} : {1}".format(st, len(test_dict[st])))"""

In [5]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 4


In [7]:
"""model = keras.models.\
load_model("/kaggle/input/keras_base_scs_l1_l2/tensorflow2/default/2/keras_base_spinal_canal_stenosis_l1_l2.h5")"""

model_dict = {}

model_dict['spinal_canal_stenosis_l1_l2'] = keras.models.\
load_model("/kaggle/input/keras_base_scs_l1_l2/tensorflow2/default/2/keras_base_spinal_canal_stenosis_l1_l2.h5")

model_dict['spinal_canal_stenosis_l2_l3'] = keras.models.\
load_model("/kaggle/input/keras_base_spinal_canal_stenosis_l2_l3/tensorflow2/default/1/keras_base_spinal_canal_stenosis_l2_l3.h5")

model_dict['spinal_canal_stenosis_l3_l4'] = keras.models.\
load_model("/kaggle/input/keras_base_spinal_canal_stenosis_l3_l4/tensorflow2/default/1/keras_base_spinal_canal_stenosis_l3_l4.h5")

model_dict['spinal_canal_stenosis_l4_l5'] = keras.models.\
load_model("/kaggle/input/keras_base_spinal_canal_stenosis_l4_l5/tensorflow2/default/1/keras_base_spinal_canal_stenosis_l4_l5.h5")

model_dict['spinal_canal_stenosis_l5_s1'] = keras.models.\
load_model("/kaggle/input/keras_base_spinal_canal_stenosis_l5_s1/tensorflow2/default/1/keras_base_spinal_canal_stenosis_l5_s1.h5")


In [8]:
def read_and_parse_dicom_files_for_inf(full_file_path):
    tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(full_file_path)
    sp = tf.strings.split(tf.gather(tf.strings.split(full_file_path, 'images/'), 1), '/')
    N = tf.size(sp)
    LEN = tf.strings.length(tf.gather(sp, 0))+tf.strings.length(tf.gather(sp, 2))
    
    # Add missing file metadata to avoid warnnigs flooding
    if   LEN==12: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==13: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==14: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==15: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==16: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==17: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==18: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x98\x00\x00\x00\x02\x00\x01\x00')
    
    #image_bytes = tf.io.read_file(full_file_path)
    #image = tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.float32)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    image = tf.image.resize(image, (128,128))
    return tf.squeeze(image)

In [9]:
vfunc = np.vectorize(read_and_parse_dicom_files_for_inf, otypes=[object])

def get_predictions(key, model_to_use):
    final_feature_list = vfunc(test_dict[key]).tolist()
    final = np.array(final_feature_list)
    return model_to_use.predict(final)

In [10]:
rows = {}
with strategy.scope():
    if Test:
        for key, value in model_dict.items():
            print("running for key :", key)
            y_proba = [get_predictions(st, model_dict[key]) for st in tqdm(test_dict.keys())] ## 27 min with 2 GPUs; not under strategy
            for i in range(len(y_proba)):
                rows[list(test_dict.keys())[i]+'_'+key] = np.mean(y_proba[i], axis=0)
    else:
        #y_proba = [get_predictions(st, model) for st in test_dict.keys()]
        for key, value in model_dict.items():
            y_proba = [get_predictions(st, model_dict[key]) for st in test_dict.keys()] ## 27 min with 2 GPUs; not under strategy
            for i in range(len(y_proba)):
                rows[list(test_dict.keys())[i]+'_'+key] = np.mean(y_proba[i], axis=0)

running for key : spinal_canal_stenosis_l1_l2


  0%|          | 0/450 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in read_and_parse_dicom_files_for_inf (vectorized)
  outputs = ufunc(*inputs)
/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 558ms/step


  0%|          | 1/450 [00:07<59:50,  8.00s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step


...

100%|█████████▉| 448/450 [30:04<00:08,  4.20s/it]

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 536ms/step


100%|█████████▉| 449/450 [30:08<00:04,  4.31s/it]

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 493ms/step


100%|██████████| 450/450 [30:13<00:00,  4.03s/it]


In [ ]:
#y_proba = (Parallel(n_jobs=90)(delayed(get_predictions)(st, model) for st in tqdm(test_dict.keys())))

In [ ]:
"""rows = {}

for i in range(len(y_proba)):
    rows[list(test_dict.keys())[i]+'_spinal_canal_stenosis_l1_l2'] = np.mean(y_proba[i], axis=0)"""

In [11]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'

In [12]:
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.555555, 0.222222, 0.222222])
        else:
            pass
            """for st in test_dict.keys():
                if c+'_'+v == 'spinal_canal_stenosis_l1_l2':
                    pass
                else:
                    rows[st+'_'+c+'_'+v] = np.array([0.555555, 0.222222, 0.222222])"""

In [13]:
if Test:
    for row_id, feature in tqdm(rows.items()):
        feature_set_reshaped = feature.reshape(1, -1)
        predictions = np.ascontiguousarray(feature_set_reshaped)
        df = pd.DataFrame(predictions, columns=severity_levels)
        df.insert(loc=0, column='row_id', value=row_id)
        submission = pd.concat([submission,df]).reset_index(drop=True)

    i = submission[(submission.row_id == 'samples')].index
    submission = submission.drop(i).reset_index(drop=True)
else:
    for row_id, feature in rows.items():
        feature_set_reshaped = feature.reshape(1, -1)
        predictions = np.ascontiguousarray(feature_set_reshaped)
        df = pd.DataFrame(predictions, columns=severity_levels)
        df.insert(loc=0, column='row_id', value=row_id)
        submission = pd.concat([submission,df]).reset_index(drop=True)

    i = submission[(submission.row_id == 'samples')].index
    submission = submission.drop(i).reset_index(drop=True)

100%|██████████| 11250/11250 [00:09<00:00, 1149.93it/s]


In [14]:
submission.to_csv('submission.csv', index=False)